# <a id='toc1_'></a>[Building LLM Agents](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Building LLM Agents](#toc1_)    
  - [Introduction to LLM Agents](#toc1_1_)    
    - [What Are LLM Agents?](#toc1_1_1_)    
    - [How Do LLM Agents Work?](#toc1_1_2_)    
  - [Building a Basic LLM Agent with OpenAI](#toc1_2_)    
  - [Enhancing LLM Agents with LangChain](#toc1_3_)    
  - [Multi-Agent Systems with LangGraph](#toc1_4_)    
    - [What's a DAG?](#toc1_4_1_)    
      - [Graph](#toc1_4_1_1_)    
      - [Acyclic](#toc1_4_1_2_)    
      - [Directed](#toc1_4_1_3_)    
    - [Where else are DAGs used?](#toc1_4_2_)    
    - [Why Are DAGs Important for LLM Agents?](#toc1_4_3_)    
    - [Building our own DAG - a simple multi-agent system](#toc1_4_4_)    
  - [Types of LLM Agents](#toc1_5_)    
    - [Web Scraping Agent](#toc1_5_1_)    
    - [Web Search Agent](#toc1_5_2_)    
    - [Retrieval-Augmented Generation (RAG) Agent](#toc1_5_3_)    
    - [Tool-Using Agent](#toc1_5_4_)    
  - [Putting it all together](#toc1_6_)    
  - [Conclusion](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Introduction to LLM Agents](#toc0_)

### <a id='toc1_1_1_'></a>[What Are LLM Agents?](#toc0_)
Large Language Model (LLM) agents are AI-powered entities that use natural language processing to interact with users, perform tasks, and automate workflows. These agents leverage models like OpenAI’s GPT to process and generate human-like text responses.

### <a id='toc1_1_2_'></a>[How Do LLM Agents Work?](#toc0_)
LLM agents typically function by:
1. **Receiving Input**: Users provide prompts or queries.
2. **Processing Input**: The agent structures the input and passes it to the LLM.
3. **Generating Output**: The LLM formulates a response based on context.
4. **Executing Actions**: If necessary, the agent can integrate with external tools to perform tasks beyond text generation.

## <a id='toc1_2_'></a>[Building a Basic LLM Agent with OpenAI](#toc0_)
Ensure you have the OpenAI Python library installed and use a `.env` file to set up your API key (`OPENAI_API_KEY`).

In [ ]:
# !pip install openai

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

Set up your API key and define a simple LLM agent function:


In [ ]:
import openai

# Define a simple LLM agent function
def llm_agent(prompt, model="gpt-4"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"]

## <a id='toc1_3_'></a>[Enhancing LLM Agents with LangChain](#toc0_)
LangChain is a powerful framework for building LLM-powered applications with memory, tools, and agentic capabilities.

In [ ]:
!pip install langchain

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-4")
prompt = PromptTemplate(input_variables=["question"], template="Answer the following: {question}")
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("What is LangChain?"))

## <a id='toc1_4_'></a>[Multi-Agent Systems with LangGraph](#toc0_)
LangGraph extends LangChain by enabling multi-agent interactions through DAGs (Directed Acyclic Graphs).

### <a id='toc1_4_1_'></a>[What's a DAG?](#toc0_)

A **Directed Acyclic Graph (DAG)** is a graph structure where nodes are connected by directed edges, and there are no cycles. This means that information flows in one direction without looping back.

#### <a id='toc1_4_1_1_'></a>[Graph](#toc0_)

A **graph** is a data structure consisting of nodes (or vertices) connected by edges. Graphs are widely used in computer science, mathematics, and AI to model relationships between entities.

![](../../../img/graph_edge_matrix.png)  
(Source: [HuggingFace](http://huggingface.co/blog/intro-graphml))

#### <a id='toc1_4_1_2_'></a>[Acyclic](#toc0_)

- **Cyclic vs. Acyclic Graphs**: A cyclic graph has cycles, whereas an acyclic graph does not.  

![](../../../img/a_cyclical_graphs.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))  

#### <a id='toc1_4_1_3_'></a>[Directed](#toc0_)

**Directed Graphs**: Edges have a specific direction (A → B).  

![](../../../img/directed-graph.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))

### <a id='toc1_4_2_'></a>[Where else are DAGs used?](#toc0_)

![](../../../img/airflow_dag.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))  

DAGs are widely used in streamlining data/ML operations that need to happen sequentially and at regular time intervals. Airflow is one of the most basic platforms where you can build this type of pipelines and monitor the success/failure of your pipelines. You can also schedule runs of pipelines, connect to cloud services, etc. Other schedulers used by data scientists are Prefect (recommended), Dask (for large-scale data processing), Kubeflow pipelines (can be overkill), DAGster. 

You'd do have to do some research before choosing any of these - Airflow is always a good starting point as it's open-source - but users complain that it's outdated compared to competitors.

### <a id='toc1_4_3_'></a>[Why Are DAGs Important for LLM Agents?](#toc0_)
- **Task Execution Flow**: DAGs define clear steps in multi-agent workflows.
- **Parallel Processing**: Independent tasks can run simultaneously.
- **Dependency Management**: Ensures tasks execute in the correct order.

LangGraph uses DAGs to structure multi-agent workflows, ensuring efficient and logical task progression.

### <a id='toc1_4_4_'></a>[Building our own DAG - a simple multi-agent system](#toc0_)

In [ ]:
!pip install langgraph

In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph.message import Message
from langchain.chat_models import ChatOpenAI

def agent_1(state):
    response = ChatOpenAI().predict("Provide an initial analysis on AI safety.")
    return Message(response)

def agent_2(state):
    response = ChatOpenAI().predict(f"Expand on the following: {state['message']}")
    return Message(response)

workflow = StateGraph()
workflow.add_nodes(["agent_1", "agent_2"])
workflow.add_edges([("agent_1", "agent_2")])
workflow.set_entry_point("agent_1")
workflow.run()

## <a id='toc1_5_'></a>[Types of LLM Agents](#toc0_)
LLM agents can be categorized based on their purpose and capabilities. Below are some common types with code examples:

### <a id='toc1_5_1_'></a>[Web Scraping Agent](#toc0_)

In [ ]:
import requests
from bs4 import BeautifulSoup

def web_agent(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()
    return text[:500]  # Return a snippet

print(web_agent("https://en.wikipedia.org/wiki/Artificial_intelligence"))

### <a id='toc1_5_2_'></a>[Web Search Agent](#toc0_)

### <a id='toc1_5_3_'></a>[Retrieval-Augmented Generation (RAG) Agent](#toc0_)
Uses vector databases for enhanced responses.

In [ ]:
import faiss
import numpy as np

def rag_agent(query, stored_vectors):
    index = faiss.IndexFlatL2(stored_vectors.shape[1])
    index.add(stored_vectors)
    query_vector = np.random.random((1, stored_vectors.shape[1])).astype('float32')
    distances, indices = index.search(query_vector, k=3)
    return indices

stored_vectors = np.random.random((10, 128)).astype('float32')
print(rag_agent("What is machine learning?", stored_vectors))

### <a id='toc1_5_4_'></a>[Tool-Using Agent](#toc0_)
An agent that integrates with APIs or tools (e.g., calculator, weather API).

In [ ]:
import requests

def weather_agent(city):
    api_key = "your_api_key_here"
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"
    response = requests.get(url)
    return response.json()["current"]["temp_c"]

print(weather_agent("New York"))

## <a id='toc1_6_'></a>[Putting it all together](#toc0_)

In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph.message import Message
from langchain.chat_models import ChatOpenAI

def agent_1(state):
    response = ChatOpenAI().predict("Summarize the latest AI research.")
    return Message(response)

def agent_2(state):
    response = ChatOpenAI().predict(f"Expand on this summary: {state['message']}")
    return Message(response)

workflow = StateGraph()
workflow.add_nodes(["agent_1", "agent_2"])
workflow.add_edges([("agent_1", "agent_2")])
workflow.set_entry_point("agent_1")
workflow.run()

## <a id='toc1_7_'></a>[Conclusion](#toc0_)
- **LLM agents** enable intelligent text-based automation.
- **LangChain** enhances LLMs with tools, memory, and structured pipelines.
- **LangGraph** facilitates multi-agent workflows for complex AI interactions.

With these tools, you can build powerful AI-driven applications. 🚀